In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier

In [ ]:
# read in data
df = pd.read_csv ('train.csv')
df1 = pd.read_csv('test.csv')

In [ ]:
# Impute missing values using mode
imp_freq = SimpleImputer(strategy="most_frequent")
i = 0
dele_list = []
ave_list = []
while i < 1205:
  name = 'x'+str(i+1)
  if df.iloc[:,i+3].isnull().sum() >= 17000:
    dele_list.append(name)
    i+=1
    continue
  value = df.iloc[:,i+3].values.reshape(-1,1)
  imp = imp_freq.fit_transform(value)
  col = df.iloc[:,i+3].dropna()
  values, counts = np.unique(col, return_counts=True)
  freq_ind = np.argmax(counts)
  ave_list.append(values[freq_ind])
  df.iloc[:,i+3] = imp
  i += 1

print(len(ave_list))
print(len(dele_list))

1064
141


In [ ]:
# data drop deleting columns
df.drop(columns=dele_list, inplace=True)
df1.drop(columns=dele_list, inplace=True)

# imputation for test data set
i = 0
while i < len(ave_list):
  name = 'x'+str(i+1)
  df1.iloc[:, i+3].fillna(ave_list[i], inplace=True)
  i += 1

df1["x80"] = df["x80"].map({1:0, 2:0, 4:0, 3:1})
df["x80"] = df["x80"].map({1:0, 2:0, 4:0, 3:1})

In [ ]:
col_cat_list = [16,17,20,21,78,79,81,82,83,84,85,86,87,148,149,150,196,197,
         198,199,200,213,303,304,305,322,472,474,476,536,537,548,573,574,
         580,581,587,588,634,635,639,640,653,
         654,659,661,664,668,677,678,686,690,703,707,
         754,766,767,781,785,787,790,804,853,
         872,874,877,891,895,896,897,911,915,919,928,
         929,930,944,948,952,965,968,970,974,978,980,981,984,985,
         988,990,994,998,1000,
         1033,1034,1039,1044,1047,
         1051,1054,1110,
         1111,1133,1135,
         1144,1175,1180,1184,1202]


print(col_cat_list)
# 164, 
col_list = []
for i in range(len(col_cat_list)):
  col_list.append('x' + str(col_cat_list[i]))

for col in col_list:
    df[col] = df[col].astype('category',copy=False)
    df1[col] = df1[col].astype('category',copy=False)

[16, 17, 20, 21, 78, 79, 81, 82, 83, 84, 85, 86, 87, 148, 149, 150, 196, 197, 198, 199, 200, 213, 303, 304, 305, 322, 472, 474, 476, 536, 537, 548, 573, 574, 580, 581, 587, 588, 634, 635, 639, 640, 653, 654, 659, 661, 664, 668, 677, 678, 686, 690, 703, 707, 754, 766, 767, 781, 785, 787, 790, 804, 853, 872, 874, 877, 891, 895, 896, 897, 911, 915, 919, 928, 929, 930, 944, 948, 952, 965, 968, 970, 974, 978, 980, 981, 984, 985, 988, 990, 994, 998, 1000, 1033, 1034, 1039, 1044, 1047, 1051, 1054, 1110, 1111, 1133, 1135, 1144, 1175, 1180, 1184, 1202]


In [ ]:
y = df.iloc[:,-1]
x = df.drop(columns=['uniqueid', 'health'])
x['new1'] = np.random.uniform(0,1,17398)
x['new2'] = np.random.randint(0,2,17398)
x['new3'] = np.random.randint(low=0, high=20, size=17398)
x_columns = x.columns.values.tolist()
#print(x_columns)
x

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has po

,year,personid,x1,x2,x3,x4,x5,x6,x7,x8,...,x1199,x1200,x1201,x1202,x1203,x1204,x1205,new1,new2,new3
0,2005,3999,5.0,3.0,5.0,4.0,3.0,2.0,5.0,7.0,...,2.0,1.0,2,160,1.0,4.0,2.0,0.496833,0,5
1,2005,3997,3.0,3.0,3.0,3.0,3.0,2.0,5.0,7.0,...,2.0,1.0,2,140,3.0,5.0,2.0,0.369931,1,4
2,2005,3996,4.0,4.0,3.0,3.0,3.0,2.0,5.0,7.0,...,2.0,1.0,2,140,3.0,5.0,2.0,0.540034,0,15
3,2005,3995,3.0,2.0,3.0,3.0,3.0,2.0,5.0,7.0,...,2.0,1.0,2,150,1.0,5.0,2.0,0.146587,0,18
4,2005,3993,4.0,2.0,4.0,4.0,3.0,2.0,2.0,7.0,...,2.0,1.0,2,130,5.0,8.0,2.0,0.698672,0,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17393,2017,514,3.0,3.0,3.0,3.0,3.0,2.0,3.0,7.0,...,1.0,1.0,1,60,4.0,9.0,9.0,0.294498,0,10
17394,2017,513,3.0,3.0,4.0,4.0,3.0,2.0,5.0,7.0,...,1.0,1.0,1,80,6.0,8.0,7.0,0.098779,0,18
17395,2017,512,2.0,2.0,2.0,2.0,3.0,2.0,2.0,7.0,...,2.0,1.0,2,150,3.0,3.0,1.0,0.274683,0,0
17396,2017,511,2.0,2.0,4.0,3.0,3.0,2.0,8.0,7.0,...,2.0,2.0,1,90,3.0,4.0,7.0,0.155093,0,19


In [ ]:
clf_f_s = RandomForestClassifier(n_estimators=300, max_features=150, bootstrap=True, oob_score=True)
clf_f_s.fit(x,y)
f = clf_f_s.feature_importances_
f
print(len(f))

imp_new1 = f[-1]
imp_new2 = f[-2]
imp_new3 = f[-3]
dele_i_list = []
for i in range(len(f)):
  if f[i] <= imp_new1 and f[i] <= imp_new2 and f[i] <= imp_new3:
    dele_i_list.append(i)
print(imp_new1, imp_new2, imp_new3)

print(len(dele_i_list))
print(max(dele_i_list))
dele_i_list.remove(max(dele_i_list))
df.drop(df.columns[dele_i_list], axis=1, inplace=True)
df1.drop(df1.columns[dele_i_list], axis=1, inplace=True)

1069
0.005584362840540105 0.0009963810854787523 0.008298010909345007
787
1067


In [ ]:
x, y = df.iloc[:,:-1],df.iloc[:,-1]

In [ ]:
x

,year,personid,x1,x2,x3,x6,x13,x16,x19,x110,...,x1178,x1180,x1182,x1183,x1184,x1201,x1202,x1203,x1204,x1205
0,2005,3999,5.0,3.0,5.0,2.0,3.0,1.0,1.0,3.0,...,3.0,3.0,2,41,2,2,160,1.0,4.0,2.0
1,2005,3997,3.0,3.0,3.0,2.0,3.0,1.0,1.0,3.0,...,2.0,3.0,1,49,5,2,140,3.0,5.0,2.0
2,2005,3996,4.0,4.0,3.0,2.0,3.0,1.0,1.0,3.0,...,3.0,3.0,1,49,5,2,140,3.0,5.0,2.0
3,2005,3995,3.0,2.0,3.0,2.0,3.0,1.0,1.0,3.0,...,2.0,3.0,1,74,5,2,150,1.0,5.0,2.0
4,2005,3993,4.0,2.0,4.0,2.0,3.0,1.0,1.0,3.0,...,2.0,3.0,1,57,2,2,130,5.0,8.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17393,2017,514,3.0,3.0,3.0,2.0,3.0,1.0,1.0,3.0,...,2.0,3.0,1,68,3,1,60,4.0,9.0,9.0
17394,2017,513,3.0,3.0,4.0,2.0,3.0,1.0,1.0,3.0,...,2.0,3.0,1,68,3,1,80,6.0,8.0,7.0
17395,2017,512,2.0,2.0,2.0,2.0,3.0,1.0,1.0,3.0,...,2.0,2.0,1,52,5,2,150,3.0,3.0,1.0
17396,2017,511,2.0,2.0,4.0,2.0,3.0,1.0,1.0,3.0,...,2.0,3.0,1,73,2,1,90,3.0,4.0,7.0


In [ ]:
from pandas.core.common import random_state
from sklearn.model_selection import KFold
clf_ls = []
kf = KFold(n_splits=5, shuffle=True,random_state=1234)
kf.get_n_splits(x)

5

In [ ]:
from imblearn.over_sampling import SMOTE

clf_gb_list = []
mf_lst = [0.33, 0.36, 0.39, 0.4, 0.42, 0.45, 0.48]

for n in mf_lst:
  sum1 = 0
  sum2 = 0
  sum3 = 0
  for train_index, test_index in kf.split(x):
    gb_clf = GradientBoostingClassifier(n_estimators=100, max_depth=3, min_samples_split=18, 
                                        min_samples_leaf=14, max_features=n)
    x_train, x_test = x.iloc[train_index], x.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]

    gb_clf.fit(x_train, y_train)
    a = log_loss(y_test, gb_clf.predict_proba(x_test))
    b = gb_clf.score(x_test,y_test)
    c = gb_clf.score(x_train,y_train)
    sum1 += a
    sum2 += b
    sum3 += c
  clf_gb_list.append((n, sum3/5, sum2/5, sum1/5))
  print(clf_gb_list[-1])

(0.33, 0.5665737794988415, 0.48505619926455285, 1.1747038150019802)
(0.36, 0.5679532106194317, 0.4844238336422476, 1.174892541512071)
(0.39, 0.5682405803594155, 0.48528629907523957, 1.1757147825462733)
(0.4, 0.5698499553815152, 0.48511353837309573, 1.174500244673004)
(0.42, 0.5688297155299428, 0.48528615039993656, 1.1754467219274594)
(0.45, 0.569433259812141, 0.4858611603942208, 1.175378366610634)
(0.48, 0.5699218274978152, 0.4863782530982281, 1.1749033948238956)


In [ ]:
clf_gb_list

NameError: ignored

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
gb_clf = GradientBoostingClassifier()
gb_clf.fit(x_train, y_train)

GradientBoostingClassifier()

In [ ]:
a = log_loss(y_test, gb_clf.predict_proba(x_test))
b = gb_clf.score(x_test,y_test)
c = gb_clf.score(x_train,y_train)
print(a, b, c)

1.1773481347339556 0.4850574712643678 0.5778129041528955


In [ ]:
df2 = pd.read_csv("test.csv")

In [ ]:
pred = gb_clf.predict_proba(df1).T
pred = pd.DataFrame(pred)
pred = pred.append(df2['uniqueid'])
pred = pred.T
pred.columns = ['p1', 'p2', 'p3', 'p4', 'p5', 'uniqueid']
pred = pred[['uniqueid', 'p1', 'p2', 'p3', 'p4', 'p5']]
pred['uniqueid'] = pred['uniqueid'].astype(int)
pred
pred.to_csv('result_gb.csv', index=False)

In [ ]:
clf_gb_list

[(0.5332078476624569, 0.4822973968606384, 1.1880486378447206),
 (0.5552936812240954, 0.4843662797804892, 1.179509541939202),
 (0.5737871870734959, 0.48563082931084045, 1.1763909032345552),
 (0.590671303447292, 0.4864354930899023, 1.175422976705193),
 (0.6064059222780754, 0.4848260994538661, 1.1753916907094726)]

In [ ]:
x.columns

Index(['year', 'personid', 'x1', 'x2', 'x3', 'x6', 'x13', 'x16', 'x19', 'x110',
       ...
       'x1178', 'x1180', 'x1182', 'x1183', 'x1184', 'x1201', 'x1202', 'x1203',
       'x1204', 'x1205'],
      dtype='object', length=282)

In [ ]:
col_cat_list = [16,17,20,21,78,79,81,82,83,84,85,86,87,148,149,150,196,197,
         198,199,200,213,303,304,305,322,332,434,435,444,472,474,476,536,537,548,573,574,
         576,580,581,583,587,588,590,634,635,636,637,639,640,653,
         654,659,661,664,668,677,678,686,690,703,707,
         754,766,767,780,781,785,787,790,804,805,811,837,853,
         867,868,872,874,877,891,892,895,896,897,911,915,919,928,
         929,930,944,948,952,965,968,970,971,974,978,980,981,984,985,
         988,990,991,994,998,1000,1001,1004,1010,1011,1014,1015,
         1018,1020,1023,1024,1027,1031,1033,1034,1039,1044,1047,
         1051,1054,1058,1061,1065,1068,1072,1075,1079,1082,1088,1110,
         1111,1133,1135,
         1144,1175,1180,1184,1202]


print(col_cat_list)
# 164, 
col_list = []
for i in range(len(col_cat_list)):
  col_list.append('x' + str(col_cat_list[i]))

for col in col_list:
    df[col] = df[col].astype('category',copy=False)
    df1[col] = df1[col].astype('category',copy=False)